In [89]:
import os, re
from collections import Counter
from tqdm import tqdm
from collections import OrderedDict
from bs4 import BeautifulSoup
import warnings
from bs4 import XMLParsedAsHTMLWarning
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning, module='bs4')
from pydub import AudioSegment

Транслитерируем и чистим

In [90]:
TRANSLIT_TABLE = {
    "-": "",
    "=": "",
    "`": "'",
    "’": "'",
    "_": "",
    "(": "",
    ")": "",
    "∅": "",
    "⊘": "",
    "ịa": "я",
    "ː" : "",
    ":" : "",
    "ń": "ɲ",
    "ǯ": "ǯ",
    "с̆": "č",
    "š": "š",
    "s̆": "š",
    "ǯo" : "дё",
    "ǯe" : "де",
    "ǯi" : "ди",
    "ǯu" : "дю",
    "ǯa" : "дя",
    "d'o" : "дё",
    "d'a" : "дя",
    "d'e" : "де",
    "d'u" : "дю",
    "d'i" : "ди",
    "ǯд" : "дд",
    "ǯ" : "дь",
    "d'" : "дь",
    "di" : "ды",
    "ti" : "ты",
    "ja" : "я",
    "je" : "е",
    "jo" : "ё",
    "ju" : "ю",
    "n'a" : "ня",
    "n'o" : "нё",
    "n'e" : "не",
    "n'i" : "ни",
    "n'u" : "ню",
    "ɲa" : "ня",
    "ɲo" : "нё",
    "ɲi" : "ни",
    "ɲe" : "не",
    "ɲu" : "ню",
    "ɲ" : "нь",
    "'u" : "ю",
    "'a" : "я",
    "'e" : "е",
    "'i" : "и",
    "'u" : "ю",
    "'" : "ь",
    "zh" : "ж",
    "a" : "а",
    "b" : "б",
    "č" : "ч",
    "c" : "ц",
    "d" : "д",
    "e" : "э",
    "f" : "ф",
    "g" : "г",
    "γ" : "г",
    "ɣ" : "г",
    "h" : "х",
    "i" : "и",
    "j" : "й",
    "k" : "к",
    "q" : "к",
    "l" : "л",
    "m" : "м",
    "n" : "н",
    "ŋ" : "ӈ",
    "o" : "о",
    "ö" : "ё",
    "p" : "п",
    "r" : "р",
    "s" : "с",
    "š" : "ш",
    "t" : "т",
    "u" : "у",
    "ü" : "ю",
    "v" : "в",
    "w" : "в",
    "x" : "х",
    "χ" : "х",
    "y" : "ы",
    "ɨ" : "ы",
    "ï" : "ы",
    "ə" : "ы",
    "z" : "з",
    "ž" : "ж"}
tr_pairs = OrderedDict(TRANSLIT_TABLE)

In [73]:
def lat2cyr(text):
    text = text.lower().strip()
    for lat, cyr in tr_pairs.items():
        text = text.replace(lat,cyr)
    return text

In [39]:
# Получаем soup файла
def get_soup(filename):
    with open('../our/'+filename) as f:
        file = f.read()
    soup = BeautifulSoup(file)
    return soup

In [19]:
# Получаем таймкоды границ
def get_timecodes(soup):
    timecodes = soup.find_all('time_slot')
    ts_msec = {}
    for timecode in timecodes:
        if timecode.get('time_value'):
            ts_msec[timecode.get('time_slot_id')] = int(timecode.get('time_value'))
    return ts_msec

Вот эти файлы надо обработать

In [20]:
filenames = os.listdir('../our')
eafs = [file for file in filenames if file.endswith('.eaf')]
eafs

['eve_KEP_20230808_biography.eaf',
 'eve_KEP_making sledge.eaf',
 'eve_KEP_20230816_upside down.eaf',
 'eve_KEP_first bear.eaf',
 'eve_TJE_deer management.eaf',
 'eve_KEP_one day from a shepherd life.eaf',
 'eve_KEP_alive little bear.eaf']

Находим фразы и их границы в eaf файлах

In [97]:
files_phrases_tc = []

for filename in tqdm(eafs):
    wavname = filename[:-4]+'.wav'
    soup = get_soup(filename)
    ts_msec = get_timecodes(soup)
    
    ann_ref = 0
    ann_phrase = {}
    for tier in soup.find_all('tier'):
        if tier.get('tier_id').startswith('transcription'):
            anns = tier.find_all('alignable_annotation')
            for ann in anns:
                ann_ts_1 = ann.get('time_slot_ref1')
                ann_ts_2 = ann.get('time_slot_ref2')
                phrase = ann.find('annotation_value').text
                checked = lat2cyr(re.sub(' +', ' ', phrase.lower().strip()))
                checked = re.sub('[.,!?]', '', checked)
                files_phrases_tc.append([wavname,
                                    phrase.strip(),
                                    checked,
                                    ts_msec[ann_ts_1],
                                    ts_msec[ann_ts_2]])

100%|█████████████████████████████████████████████| 7/7 [00:00<00:00, 11.99it/s]


In [91]:
spans = pd.DataFrame(files_phrases_tc, columns = ['filename', 'phrase', 'checked', 'start', 'end'])
file_id = {}
for id, filename in enumerate(spans['filename'].unique()):
    file_id[filename] = str(id)
spans['file_id'] = spans['filename'].apply(lambda x: file_id[x])
spans['length'] = spans['end'] - spans['start']
spans

,filename,phrase,checked,start,end,file_id,length
0,eve_KEP_20230808_biography.wav,bi kuŋa-rəp-č ərəγər bi-ši-wu etike-nu-mi,би куӈарыпч ырыгыр бишиву этыкэнуми,90,3790,0,3700
1,eve_KEP_20230808_biography.wav,or-dula bi-ǯ-ǯot-tə-m,ордула биддёттым,6730,8370,0,1640
2,eve_KEP_20230808_biography.wav,nan rɨbalka-ǯ-ǯot-t-u gobdi-ǯ-ǯot-t-u,нан рыбалкаддётту гобдыддётту,8705,11695,0,2990
3,eve_KEP_20230808_biography.wav,kam-gi-ǯ-ǯot-t-u,камгиддётту,11930,13610,0,1680
4,eve_KEP_20230808_biography.wav,nan,нан,14920,15650,0,730
...,...,...,...,...,...,...,...
124,eve_KEP_alive little bear.wav,tumuje-ǯi-p ora-m nɨktɨk nɨkərə-w-dɨn,тумуедип орам ныктык ныкырывдын,24150,26782,5,2632
125,eve_KEP_alive little bear.wav,amaški ŋənə-d-∅-du noŋa-n ačča,амашки ӈыныдду ноӈан ачча,27920,29940,5,2020
126,eve_KEP_alive little bear.wav,i-le utə ur-ri-n,илэ уты уррин,30010,31460,5,1450
127,eve_KEP_alive little bear.wav,nan teleŋ-ni-wu etiken-teki,нан тэлэӈниву этыкэнтэки,33087,35177,5,2090


In [96]:
spans.to_csv('even_spans_hse.csv', index=False)

In [85]:
spans['length'].describe()

count     129.000000
mean     3124.209302
std      1541.288638
min       510.000000
25%      1800.000000
50%      2986.000000
75%      3980.000000
max      7350.000000
Name: length, dtype: float64

In [87]:
spans['length'].sum() / 60000

6.71705

Режем файлы, сохраняем метаданные

In [93]:
file_phrase = []
filenames = spans['filename'].unique()

for filename in tqdm(filenames):
    audio = AudioSegment.from_wav("../our/"+filename)
    for ann_id, row in enumerate(spans[spans['filename'] == filename].iterrows()):
        row = row[1]
        file_num = '_000'[:-len(row['file_id'])]+row['file_id']
        ann_num = '_000'[:-len(str(ann_id+1))]+str(ann_id+1)
        newfilename = 'eve_hse'+file_num+ann_num+'.wav'
        
        newAudio = audio[row['start']:row['end']]
        newAudio.export('../dataset_hse/'+newfilename, format="wav")
        
        file_phrase.append([filename, newfilename, row['checked']])

100%|█████████████████████████████████████████████| 6/6 [00:02<00:00,  2.46it/s]


In [94]:
df = pd.DataFrame(file_phrase, columns = ['file_orig', 'file', 'text'])
df

,file_orig,file,text
0,eve_KEP_20230808_biography.wav,eve_hse_000_001.wav,би куӈарыпч ырыгыр бишиву этыкэнуми
1,eve_KEP_20230808_biography.wav,eve_hse_000_002.wav,ордула биддёттым
2,eve_KEP_20230808_biography.wav,eve_hse_000_003.wav,нан рыбалкаддётту гобдыддётту
3,eve_KEP_20230808_biography.wav,eve_hse_000_004.wav,камгиддётту
4,eve_KEP_20230808_biography.wav,eve_hse_000_005.wav,нан
...,...,...,...
124,eve_KEP_alive little bear.wav,eve_hse_005_009.wav,тумуедип орам ныктык ныкырывдын
125,eve_KEP_alive little bear.wav,eve_hse_005_010.wav,амашки ӈыныдду ноӈан ачча
126,eve_KEP_alive little bear.wav,eve_hse_005_011.wav,илэ уты уррин
127,eve_KEP_alive little bear.wav,eve_hse_005_012.wav,нан тэлэӈниву этыкэнтэки


In [95]:
df[['file', 'text']].to_csv('even_speech_hse.csv', index=False)